In [1]:
import numpy as np
import pandas as pd
from nilearn import image
from nilearn import plotting
import matplotlib as plt
import sys
import os
import fnmatch
import warnings
warnings.filterwarnings("ignore")
import json

## Lables of the ADNI dataset

In [ ]:
alldata = pd.read_csv("./Data/ADNI_Lables_Total.csv")               # when the lables of the ADNI dataset are saved in csv file in folder named "Data"
dict_imageStatus = dict(zip(alldata.Subject, alldata.Status))       # Saving the dataset in a dictionray with key: patients' IDs and value: patients' Alzheimer status

## ADNI MRI pictures processing

In [ ]:
rootPath = './Data'              # when the MRI images are saved in a folder named "Data"
pattern = '*.nii'
                                       
[processedImgNo, counter] = [1, 1]    
[batchNo, batchSize] = [1, 10]

X = Y = np.array([])                                        # X: matrix of features (number of samples (m) by number of features (n = 256 * 256 * 170))
                                                            # Y: matrix of lables (number of samples (m) by 1)
# XY_Values: folder for saving X matrix and Y matrix
if not os.path.exists('XY_Values'):
    os.makedirs('XY_Values')
    
for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        dict_imageData = dict()  # a dicionary from subject_ID to pixels of it's MRI picture (picture: a 3rd order tensor)
        dataFiles = [] 
        dataFiles = (os.path.join(root, filename)) 
        key = str(dataFiles.split('\\')[1])                 # subject_ID
        img = image.load_img(dataFiles) 
        val = img.get_data()[:, :, :]                         # getting the images data
        
        # padding val matrix with zeros to enlarge them upto the size 256*256*170
        [xshape, yshape, zshape] = [val.shape[0], val.shape[1], val.shape[2]]
        [xpad, ypad, zpad] = [int((256-xshape)*0.5), int((256-yshape)*0.5), int((170-zshape)*0.5)]
        valPadded = np.pad(val, ((xpad, xpad), (ypad, ypad), (zpad, zpad)), 'constant')
        
        val = np.reshape(valPadded, [-1])                   # flattening the images data into a row vector
        dict_imageData [key] = (np.array(val))
                            
        for key, value in dict_imageData.items():           # looping through dictionary of image data
            if key in dict_imageStatus.keys():
                if X.size != 0:
                        X = np.vstack([X, (value)])
                        Y = np.vstack([Y, dict_imageStatus[key]])
                else:
                        X = (value)
                        Y = dict_imageStatus[key]
                        
                if counter % batchSize == 0:
                    counter = 1
                    assert(X.shape[0] == Y.shape[0])
                    assert(X.shape[1] == 256 * 256 * 170 )  
                    np.save(os.path.join('XY_Values', 'XValues' + str(batchNo) + '.npy'), X)
                    np.save(os.path.join('XY_Values', 'YValues' + str(batchNo) + '.npy'), Y)
                    print('No. of images processed: ', processedImgNo , ', No. of batches saved: ', batchNo)
                    batchNo = batchNo + 1
                    X = np.array([]) 
                    Y = np.array([]) 
                else: 
                    counter = counter + 1
                processedImgNo = processedImgNo + 1 

No. of images processed:  10 , No. of batches saved:  1
